In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, Activation, MaxPooling1D, Dense, Dropout, Flatten
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers

In [2]:
#Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
#Extract Train and Test Data.
df_tr = pd.read_csv('Data\Intrusion-Detection\Train_data.csv',na_values='?',header=0)
df_ts = pd.read_csv('Data\Intrusion-Detection\Test_data.csv',na_values='?',header=0)

#Drop Duplicates
df_tr.drop_duplicates(keep='first')
df_ts.drop_duplicates(keep='first')

#Drop NA
df_tr.dropna()
df_ts.dropna()

#Drop ClassLabel column from input array X and Create output array y of Classlabel.
Xtr = np.array(df_tr.drop('xAttack',axis=1))
ytr = np.array(df_tr['xAttack'])
Xts = np.array(df_ts.drop('xAttack',axis=1))
yts = np.array(df_ts['xAttack'])

#Encode
le = preprocessing.LabelEncoder()
lb = preprocessing.LabelBinarizer()

Xtr[:,1] = le.fit_transform(Xtr[:,1])
Xts[:,2] = le.fit_transform(Xts[:,2])
Xts = np.delete(Xts,0,axis=1)

ytr = le.fit_transform(ytr)
lb.fit_transform(ytr)

yts = le.fit_transform(yts)
lb.fit_transform(yts)

In [5]:
print(Xtr.shape)
print(ytr.shape)

print(Xts.shape)
print(yts.shape)

In [6]:
#Nomralize the data
Xtr_n = preprocessing.normalize(Xtr)
Xts_n = preprocessing.normalize(Xts)

In [9]:
ncomp =20
# Construct the PCA object
pca = PCA(n_components=ncomp, 
          svd_solver='randomized', whiten=True)

# Fit the PCA components on the entire dataset
pca.fit(Xtr_n)
Ztr = pca.transform(Xtr_n)
Zts = pca.transform(Xts_n)

In [10]:
K.clear_session()

In [11]:
nout = int(np.max(ytr)+1) 
nin = Ztr.shape[1]

Ztr_new = Ztr.reshape((Ztr.shape[0], Ztr.shape[1], 1))
Zts_new = Zts.reshape((Zts.shape[0], Zts.shape[1], 1))

model = Sequential()
model.add(Conv1D(kernel_size=5, filters=64, input_shape=(Ztr.shape[1],1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=nout, activation='softmax', name='output'))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

hist = model.fit(Ztr_new, ytr, batch_size=100, epochs=5,validation_data=(Zts_new,yts))

In [12]:
nout = int(np.max(ytr)+1) 
nin = Ztr.shape[1]

Ztr_new = Ztr.reshape((Ztr.shape[0], Ztr.shape[1], 1))
Zts_new = Zts.reshape((Zts.shape[0], Zts.shape[1], 1))

model = Sequential()
model.add(Conv1D(kernel_size=5, filters=64, input_shape=(Ztr.shape[1],1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=nout, activation='softmax', name='output'))

opt = optimizers.Adam(lr=0.001)
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

hist = model.fit(Ztr_new, ytr, batch_size=100, epochs=5,validation_data=(Zts_new,yts))

In [13]:
tr_accuracy = hist.history['acc']
val_accuracy = hist.history['val_acc']

plt.plot(tr_accuracy)
plt.plot(val_accuracy)
plt.grid()
plt.xlabel('epochs')
plt.ylabel('accuarcy')
plt.legend(['training accuracy', 'validation accuracy'])

In [14]:
maxTr = np.argmax(tr_accuracy)
maxTs = np.argmax(val_accuracy)
print('Max Training Accuracy obtained:',tr_accuracy[maxTr])
print('Max Test Accuracy obtained:',val_accuracy[maxTs])